Welcome!  If you are new to Google Colab/Jupyter notebooks, you might take a look at [this notebook](https://colab.research.google.com/notebooks/basic_features_overview.ipynb) first.

**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](http://underactuated.csail.mit.edu/lqr.html).**

# Notebook Setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- import packages used throughout the notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast (even on Colab) because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='560c2adace05eb20ebd78377582015d5b2d3859a', drake_version='0.27.0', drake_build='release')

# Imports.
import numpy as np
from IPython.display import display
from ipywidgets import interact, Textarea
import pydrake.all


# Discrete-time vs continuous-time LQR

Let's compare the two solutions in our simplest of systems: the double integrator.

## Continuous time

In [ ]:
A = np.array([[0,1],[0,0]])
B = np.array([[0],[1]])
Q = np.identity(2)
R = np.identity(1)

K,S = pydrake.systems.controllers.LinearQuadraticRegulator(A,B,Q,R)
print(f"S = {S}\n")

eigenvalues, eigenvectors = np.linalg.eig(S)
print(f"eig(S) = {eigenvalues}")


## Discrete time

In [ ]:
# discrete time

Stext = Textarea(value="", description="S: ", layout={'height':'40px'}, style={'description_width':'initial'})
eigtext = Textarea(value="", description="eig(S): ", layout={'height':'30px'}, style={'description_width':'initial'})
display(Stext)
display(eigtext)

def update(h=1):
    A = np.array([[1,h],[0,1]])
    B = np.array([[0],[h]])
    Q = h*np.identity(2)
    R = h*np.identity(1)

    K,S = pydrake.systems.controllers.DiscreteTimeLinearQuadraticRegulator(A,B,Q,R)
    Stext.value = np.array2string(S, formatter={'float': lambda x: "{:5.2f}".format(x)})

    eigenvalues, eigenvectors = np.linalg.eig(S)
    eigtext.value = np.array2string(eigenvalues, formatter={'float': lambda x: "{:5.2f}".format(x)})

interact(update, h=(0.01,1,.01));

There are a few things to notice here.  First, the discrete-time solution converges to the continuous-time solution as the timestep goes to zero.  Second, the cost-to-go is always higher in the discrete-time version: you should think of the discrete time as adding an additional constraint that the control decision can only be changed once per timestep.  Adding constraints can only increase the total cost.

### Side note: Algebraic solution

In the DP chapter, I was able to give a nice closed-form solution for the continuous-time case:$$S = \begin{bmatrix} \sqrt{3} & 1 \\ 1 & \sqrt{3} \end{bmatrix}.$$  The discrete-time case is not as clean.  Even when $h=1$, the discrete-time Riccati equation, using $$S = \begin{bmatrix} a & b \\ b & c \end{bmatrix},$$ results in three equations: \begin{gather*} b^2 = 1+c \\ 1 + c + bc = a + ac \\ c^2 = 2b + a + ac. \end{gather*}  With a little work, you can reduce this to to a quadratic equation in $b$, with a solution, $b = \frac{1}{4}(1 + \sqrt{21} + \sqrt{2(3+\sqrt{21})}).$  Not so nice! 

### Side note: Exact integration

One can also use the exact integral of the linear system $e^Ah$ in the discretization, instead of the Euler discretization.  It doesn't change the basic observation.